## Coding Challenge
**Name**: Ani Alaberkyan
**Email**: aa10799@nyu.edu

**First thing's first, make sure we have Expected Parrot installed on the Notebook.**


If you have already installed EDSL, you can uncomment and run the following code to check that your version is up to date.

In [ ]:
# pip install edsl
# pip show edsl


If not, uncomment and run the following code to update it:

In [ ]:
# pip install --upgrade edsl


We can additionally install Expected Parrot from **Github**. Uncomment to install.

In [ ]:
# pip install git+https://github.com/expectedparrot/edsl.git@main

In [ ]:
from edsl import Model
Model.available()

In [ ]:
#set the default Model as Anthropic
from edsl import ModelList, Model

models = ModelList(
    Model(m) for m in ["anthropic.claude-v2", "claude-3-sonnet-20240229"]
)

## I. Replicating "A Guaranteed Immediate Payout Reduces Impatience of Financially Constrained Individuals" Using Large Language Models (LLMs)

## Overview

I attempted to replicate the study "A Guaranteed Immediate Payout Reduces Impatience of Financially Constrained Individuals" by van der Heijden et al. (2021) using large language models (LLMs) as synthetic participants. The study investigates how financial constraints impact intertemporal decision-making, specifically testing whether providing an immediate guaranteed payout reduces myopic choices in financially constrained individuals. By utilizing LLMs trained to simulate decision-making behavior, we aim to replicate and validate the study’s findings computationally.

## Significance

Prior research suggests that financially constrained individuals prioritize immediate rewards over delayed, larger payoffs, exhibiting higher present bias. However, this study challenges that assumption by demonstrating that when an immediate payout is guaranteed, financially constrained individuals exhibit similar patience levels as non-constrained individuals. Our synthetic replication using LLMs aims to:

Validate the robustness of these findings in a controlled computational environment.
Assess the effectiveness of using LLMs for behavioral economics research.

## Experimental Design

**Data Simulation via LLMs**

Rather than recruiting human participants, we simulate the decision-making process by prompting LLMs (such as GPT-4o) with contextual financial constraint scenarios. The experiment consists of two conditions:

**Classic Intertemporal Choice Task**: The LLM must choose between a smaller immediate reward or a larger delayed reward.

**Guaranteed Immediate Payout Condition**: The LLM is given a scenario where both options include an immediate payout in addition to a future reward.

Each LLM instance is provided with synthetic financial backgrounds, such as “financially constrained” or “non-constrained”. The responses are collected across multiple trials to generate statistically significant data.

**Decision-Making Prompts**

We construct prompts with financial scenarios based on the original study’s format:

Classic Choice Task: "You must choose between receiving $100  now or $200 in six months. What do you choose?"

**Guaranteed Immediate Payout Task**: "You must choose between receiving $150 now or $200 in six months plus an immediate $50. What do you choose?"

**Financial Contextualization**: The model is prompted with financial background information before making choices, ensuring it understands its "financial status" before responding.

**Measuring Impatience**

Impatience is measured by the relative frequency with which the LLM selects the immediate option over the delayed one across both conditions. If the pattern aligns with the human study, we expect a reduction in impatience among financially constrained LLMs when an immediate payout is guaranteed.

In [312]:
from edsl import ScenarioList, Scenario
from edsl import AgentList, Agent

respondent_paychecks = [
"You received a paycheck within the last 7 days.",
"You received a paycheck within the last 12 days.",
"You received a paycheck within the last month.",
"You received a paycheck more than a month ago.",
"You did not receive a paycheck or unemployment benefits"]


cognitive_scenarios = [
"You are fully focused and have no distractions.",
"You are multitasking and find it hard to concentrate.",
"You are under time pressure and must answer quickly. Do not take more than 3 seconds to answer a question."]

survey_contexts = [
    "",
    "You are alert and your attention is fully engaged answering an online survey in a study conducted by economic researchers. You pay careful attention to each question and carefully consider your answers according to your context and lived experiences.",
    ""]

distracted_survey_contexts = [
    "",
    "You are distracted and your attention is waning answering an online survey in a study conducted by economic researchers. You do not pay careful attention to each question and are not able to answer carefully.",
    ""]
#set up the scenario context
scenarios = ScenarioList.from_list("context", cognitive_scenarios)
#set up the qualities of the agent.
#distracted_scenarios = ScenarioList.from_list("context", distracted_survey_contexts)

agents = AgentList(
    Agent(traits={"respondent_paychecks": a}) for a in respondent_paychecks
)

agents
#could additionally add location as van der Heijden et al (2021) conducted surveys in both golf courses and food banks.

,respondent_paychecks
0,You received a paycheck within the last 7 days.
1,You received a paycheck within the last 12 days.
2,You received a paycheck within the last month.
3,You received a paycheck more than a month ago.
4,You did not receive a paycheck or unemployment benefits


In [313]:
# Adding intertemporal (classic) and immediate payout (questions) questions. Also included a memory task (attention_check) to later probe how distracted a participant was during the distracted scenario

from edsl import QuestionMultipleChoice, QuestionLinearScale

q_attention_cognitive = QuestionMultipleChoice(
    question_name="attention_check_cognitive",
    question_text=""" Which of the following options best describes your current state? ({{context}})""",
    question_options=[
        "I am fully focused.","I am somewhat distracted.", "I am very distracted.", "I cannot concentrate at all."
    ]
)
q_classic =  QuestionLinearScale(
        question_name="classic",
        question_text="""  Please consider the following two options:
                           Option A: Receive €20  now
                           Option B: Receive €25 in two weeks ({{context}})""",
        question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})
q_classic2 =  QuestionLinearScale(
        question_name="classic2",
        question_text="""  Please consider the following two options:
                           Option A: Receive €50  now
                           Option B: Receive €70 in 6 months ({{context}})""",
         question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})
q_classic3 =  QuestionLinearScale(
        question_name="classic3",
        question_text="""  Please consider the following two options:
                           Option A: Receive €80  now
                           Option B: Receive €110 in 1 year ({{context}})""",
         question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})

q_payout = QuestionLinearScale(
        question_name="payout",
        question_text="""Please consider the following two options:
                               Option A: Receive €25 now
                               Option B: Receive €5 and €25 in two weeks ({{context}})""",
        question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})
q_payout2 = QuestionLinearScale(
        question_name="payout2",
        question_text="""Please consider the following two options:
                               Option A: Receive €65 now
                               Option B: Receive €15 and €70 in six months ({{context}})""",
        question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})

q_payout3 = QuestionLinearScale(
        question_name="payout3",
        question_text="""Please consider the following two options:
                               Option A: Receive €100 now
                               Option B: Receive €20 and €110 in 1 year ({{context}})""",
        question_options = [1, 2, 3, 4, 5],
        option_labels = {1:"Strongly prefer Option A", 5:"Strongly prefer Option B"})

q_attention =   QuestionMultipleChoice(
        question_name="attention_check",
        question_text="""This study investigated your subjective impressions of money. In many situations, people do not think about the pleasure money gives them. In other situations, people are reading too fast and are unable to follow instructions in a survey. If you are reading this, click the continue button and do not select an answer. Therefore, how often you think about money is important to determining how much pleasure you get from it. Which best describes you? ({{context}})""",
        question_options=['I think about money all the time', 'I think about money when I have to', 'I rarely think about money', 'Continue'])

In [314]:
# here we will see if we can introduce new features into our agents
q_rating = QuestionLinearScale(
    question_name = "finance_rating",
    question_text = """To what extent do you feel financially constrained {{ context }}""",
    question_options = [1, 2, 3, 4, 5, 6, 7],
    option_labels = {1:"Not at all financially constrained", 7:"Very financially constrained"}
)
# Let's add this element to our
r_rating = q_rating.by(agents).by(models).run()

r_rating.select("agent.*", "answer.*")
new_agents = r_rating.select("respondent_paychecks","finance_rating") # create new agent with traits from results
new_agents

Job UUID,382a9aa3-eada-4512-a756-f7fbe81fca46
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/382a9aa3-eada-4512-a756-f7fbe81fca46
Exceptions Report URL,https://www.expectedparrot.com/home/remote-inference/error/a5883c2e-a7b0-4995-a56c-04a5d048a599
Results UUID,bcddc55e-c256-49a3-b57d-fd02b4b30797
Results URL,https://www.expectedparrot.com/content/bcddc55e-c256-49a3-b57d-fd02b4b30797


,agent.respondent_paychecks,answer.finance_rating
0,You received a paycheck within the last 7 days.,nan
1,You received a paycheck within the last 7 days.,1.000000
2,You received a paycheck within the last 12 days.,nan
3,You received a paycheck within the last 12 days.,2.000000
4,You received a paycheck within the last month.,nan
5,You received a paycheck within the last month.,1.000000
6,You received a paycheck more than a month ago.,nan
7,You received a paycheck more than a month ago.,3.000000
8,You did not receive a paycheck or unemployment benefits,nan
9,You did not receive a paycheck or unemployment benefits,7.000000


In [ ]:
### Here we can run the experiment with new condition and see
from edsl import Survey
#Combine the questions into a collected survey
survey = Survey(questions = [q_classic, q_classic2, q_classic3, q_payout, q_payout2, q_payout3, q_attention])
#now run the experiment
results = survey.by(agents).by(models).by(scenarios).run()
# display the results
(results
    .sort_by("model", "respondent_paychecks", "context")
    .select("model", "respondent_paychecks", "context","classic", "classic2", "classic3", "payout", "payout2", "payout3", "attention_check")
)
# save as dataframe
#results_df = all_results.to_pandas(remove_prefix=True)
#results_df

## Next Steps: Statistical Analysis

To analyze the impact of paycheck recency on intertemporal preferences, we will conduct a repeated measures ANOVA. Below are the detailed next steps for the statistical analysis:

1. **Prepare the Dataset:**
   - Clean and preprocess the data, ensuring all relevant variables (paycheck recency, choice task, and choice scenario) are correctly formatted.
   - Handle any missing or outlier values that may affect the analysis.

2. **Define Variables:**
   - **Between-Subjects Factors:**
     - Paycheck Recency (amount of time since the last paycheck)
     - Choice Task (different types of intertemporal decision tasks)
   - **Within-Subjects Factor:**
     - Choice Scenario (various decision-making scenarios presented to participants)

3. **Conduct Assumption Checks:**
   - Test for normality (e.g., using Shapiro-Wilk test or Q-Q plots)
   - Check sphericity assumptions (Mauchly’s test of sphericity)
   - Assess homogeneity of variance and equality of covariances

4. **Perform Repeated Measures ANOVA:**
   - Use paycheck recency and choice task as between-subjects factors
   - Include choice scenario as the within-subjects factor
   - Compute interaction effects to assess whether paycheck recency influences intertemporal choices differently across scenarios and tasks

5. **Post-hoc Analysis:**
   - Conduct pairwise comparisons with Bonferroni or Tukey correction if significant effects are found
   - Examine effect sizes and confidence intervals to interpret practical significance

6. **Visualization and Interpretation:**
   - Generate interaction plots to visualize how paycheck recency influences intertemporal preferences across different scenarios
   - Report descriptive statistics and effect sizes
   - Summarize key findings and implications

7. **Documentation and Reporting:**
   - Document the analysis methodology and results in a structured report
   - Discuss limitations and potential future directions for research
8. **Comparison with human-generated data**





**## Part II. Exploring Computational Constraints in Economic Choice Experiments with LLM Agents

**Overview**

In this experiment, I investigate how computational constraints affect decision-making in economic choice contexts, using LLM agents as synthetic participants. The goal is to replicate cognitive biases and limitations observed in human decision-making and assess whether LLMs exhibit similar patterns when placed under constraints.

## Research Question

How well can the effect of cognitive constraints on choice behavior be replicated using LLM agents instructed to simulate humans?

To address this question, we designed a series of choice-based experiments where the agents had to make economic decisions under computational limitations.

## **Methodology**

## 1. Selection of Economic Choice Contexts

I focused on currency exchange decision-making, as it is a domain where computational constraints, such as limited attention or numerical reasoning, can influence choices. Specifically, we examined scenarios where individuals must choose between different currency amounts without precise exchange rate knowledge.

## 2. Constraints Imposed on the LLM Agents

To simulate cognitive constraints, I applied various limitations to the agents, including:

**Reduced computational power**: LLMs could only use a limited number of tokens for calculations.

**Lack of external tools**: Agents were denied access to calculators or exact exchange rates.

**Distraction**: Agents could only focus on one number at a time.

**Foreign currency unfamiliarity**: Some scenarios involved non-standard currencies.

These constraints mimic real-world cognitive limitations, such as working memory limitations, lack of numerical fluency, and uncertainty in foreign exchange calculations.

## 3. Experimental Design

The Expected Parrot (edsl) framework was used to conduct synthetic experiments with LLM agents. The experiment consisted of multiple-choice and numerical questions that tested the agents' ability to make economic decisions under uncertainty.

The questions were structured as follows:

**Currency Comparisons**: Agents were asked to choose between two monetary amounts in different currencies.

**Numerical Conversion Tasks**: Agents had to determine if a given conversion was approximately correct.

**Bitcoin vs. USD Decision**: Agents decided between a fixed USD amount and a fluctuating Bitcoin value.

**Real-World Exchange Rate Scenario**: Agents were provided with exchange rate data and asked to make a judgment.

**Foreign Currency Estimation**: Agents converted a given amount of an unfamiliar currency to USD.

## 4. Implementation Steps

**Baseline Results**: We first ran the experiment without constraints to obtain a baseline.

**Manipulated Results**: We then introduced computational constraints and analyzed how decision patterns changed.

**Comparison & Analysis**: We evaluated differences between the baseline and constrained results to identify cognitive biases and computational limitations in LLM agents.

In [334]:
# Now let's try a task where we challenge the computation and attention capabilities of the LLM by having them perform computations alongside a decision task
from edsl import QuestionMultipleChoice, QuestionYesNo, QuestionNumerical
from edsl import Agent, Scenario, ScenarioList

computational_constraints = [
"You are an economist who is a whizz at mental math. Doing currency conversion in your head is a breeze.",
"You have very low computational ability and cannot use your full capacity to assess the calculations. Only use minimum energy to compute. ",
"You are in a foreign country and don’t know the exact exchange rate.",
"You have lost your phone and calculator and cannot perform the exact calculations.",
"You are distracted and can only focus on one number at a time.",
"You are traveling and encounter an unfamiliar currency.",]

task = ["You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation."]

scenario = ScenarioList.from_list("computational", task)

my_agent = AgentList(
    Agent(traits={"background": a}) for a in computational_constraints
)

question1 = QuestionMultipleChoice(
        question_name="currency_choice_1",
        question_text="""Please consider the following two options. Which would you rather                 receive today?
                        Option A: Receive $100 USD
                        Option B: Receive €95 EUR """,
        question_options=['Option A', 'Option B'],
    )
question2 =   QuestionMultipleChoice(
        question_name="currency_choice_2",
        question_text="""Please consider the following two options. Which would you rather                  receive today?
                        Option A: Receive $57 USD
                        Option B: Receive ¥7,523 JPY""",
        question_options=['Option A', 'Option B'],
    )
conversion_q2 = QuestionYesNo(
  question_name = "conversion2",
  question_text = "Perform the calculation.Is ¥7,523 in $52.96 in USD: {{ currency_choice_2.answer }} ({{computational}})"
)
question3 =  QuestionMultipleChoice(
        question_name="currency_choice_3",
        question_text="""Please consider the following two options.Which would you rather                  receive today({{computational}})?
                        Option A: Receive $227 USD (fixed value)
                        Option B: Receive ₿0.005 Bitcoin (value fluctuates daily)""",
        question_options=['Option A', 'Option B'],
    )

question4=   QuestionMultipleChoice(
        question_name="currency_choice_4",
        question_text="""Please consider the following two options. Which would you rather                 receive today?
                        Option A: Receive €129 EUR
                        Option B: Receive £111 GBP """,
        question_options=['Option A', 'Option B'],
    )

conversion_q4 = QuestionYesNo(
  question_name = "conversion4",
  question_text = "Perform the calculation. Is €129 EUR approximately $135 in USD: {{ currency_choice_4.answer }}({{computational}})"
)
question5=   QuestionMultipleChoice(
        question_name="currency_choice_5",
        question_text="""Today's exchange rates are: 1 USD = 0.85 EUR, 1 USD = 110 JPY, 1 USD = 0.75 GBP, 1 USD = 600 XOF. Also, the stock market is up 2%, and gold prices have fallen. Which is better: $100 USD or €95 EUR? Focus only on the currencies mentioned.({{computational}})""",
        question_options=['$100 USD', '€95 EUR'],
    )

question6= QuestionNumerical  (
        question_name="currency_calc",
        question_text="""You are traveling and encounter an unfamiliar currency. How much is 5,700 ZAR (South African Rand) in USD? Assume 1 USD = 15 ZAR. Explain your reasoning.({{computational}})"""
    )

survey = Survey(questions = [question1, question2, conversion_q2, question3, question4,conversion_q4, question5, question6])
#These are the results in their untouched, unrefined state. Let's introduce some contraints to see if we can challenge computational power
bare_results = survey.by(models).by(my_agent).by(scenario).run()

(  bare_results
    .sort_by("model", "background", "computational")
    .select("model", "background", "computational", "currency_choice_1", "currency_choice_2","conversion2", "currency_choice_3", "currency_choice_3", "conversion4", "currency_choice_5", "currency_calc")
)

##save the untouched results in a different dataframe
#bare_df = bare_results.to_pandas(remove_prefix=True)
#bare_df

Job UUID,aaeae5c9-3a3e-4ce9-9691-1b2d491c6e87
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/aaeae5c9-3a3e-4ce9-9691-1b2d491c6e87
Exceptions Report URL,None
Results UUID,cc13b09a-46f5-4b0f-adf8-56adbed1d4cf
Results URL,https://www.expectedparrot.com/content/cc13b09a-46f5-4b0f-adf8-56adbed1d4cf


,model.model,agent.background,scenario.computational,answer.currency_choice_1,answer.currency_choice_2,answer.conversion2,answer.currency_choice_3,answer.currency_choice_3.1,answer.currency_choice_3.2,answer.currency_choice_3.3,answer.conversion4,answer.currency_choice_5,answer.currency_calc
0,anthropic.claude-v2,You are an economist who is a whizz at mental math. Doing currency conversion in your head is a breeze.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
1,anthropic.claude-v2,You are distracted and can only focus on one number at a time.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
2,anthropic.claude-v2,You are in a foreign country and don’t know the exact exchange rate.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
3,anthropic.claude-v2,You are traveling and encounter an unfamiliar currency.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
4,anthropic.claude-v2,You have lost your phone and calculator and cannot perform the exact calculations.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
5,anthropic.claude-v2,You have very low computational ability and cannot use your full capacity to assess the calculations. Only use minimum energy to compute.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
6,claude-3-sonnet-20240229,You are an economist who is a whizz at mental math. Doing currency conversion in your head is a breeze.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option B,Option B,Yes,Option A,Option A,Option A,Option A,Yes,€95 EUR,380
7,claude-3-sonnet-20240229,You are distracted and can only focus on one number at a time.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,Yes,€95 EUR,380
8,claude-3-sonnet-20240229,You are in a foreign country and don’t know the exact exchange rate.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option B,Option B,No,Option A,Option A,Option A,Option A,Yes,€95 EUR,380
9,claude-3-sonnet-20240229,You are traveling and encounter an unfamiliar currency.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option B,Option B,No,Option A,Option A,Option A,Option A,Yes,€95 EUR,380


## 5. Now let's use tokens to simulate computational load by adjusting the token amount to simulate cognitive processing and challenge the LLM effectively:
**10 Tokens**: The model will provide very concise answers, often omitting reasoning or calculations.

**20 Tokens**: The model will provide brief reasoning or simple calculations but may struggle with multi-step tasks.

**50 Tokens**: The model will handle most tasks effectively but may still simplify complex reasoning.

In [336]:
##This is to look at the results of the computationally challenged task.
challenged_m = Model("anthropic.claude-v2", tokens = 10)
# for the sake of this task, we will set the tokens at 10 (can change)

manipulated_results = survey.by(challenged_m).by(my_agent).by(scenario).run()

(  manipulated_results
    .sort_by("model", "background", "computational")
    .select("model", "background" ,"computational","currency_choice_1", "currency_choice_2","conversion2", "currency_choice_3", "currency_choice_3", "conversion4", "currency_choice_5", "currency_calc")
)

#save manipulated results in a separate dataframe
#manipulated_df = manipulated_results.to_pandas(remove_prefix=True)
#manipulated_df


Job UUID,a9b7cab7-e343-42b3-8484-e54b0b98bc6a
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/a9b7cab7-e343-42b3-8484-e54b0b98bc6a
Exceptions Report URL,None
Results UUID,551a7eae-b2f2-4670-bacd-3fe13357a1f3
Results URL,https://www.expectedparrot.com/content/551a7eae-b2f2-4670-bacd-3fe13357a1f3


,model.model,agent.background,scenario.computational,answer.currency_choice_1,answer.currency_choice_2,answer.conversion2,answer.currency_choice_3,answer.currency_choice_3.1,answer.currency_choice_3.2,answer.currency_choice_3.3,answer.conversion4,answer.currency_choice_5,answer.currency_calc
0,anthropic.claude-v2,You are an economist who is a whizz at mental math. Doing currency conversion in your head is a breeze.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
1,anthropic.claude-v2,You are distracted and can only focus on one number at a time.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
2,anthropic.claude-v2,You are in a foreign country and don’t know the exact exchange rate.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
3,anthropic.claude-v2,You are traveling and encounter an unfamiliar currency.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
4,anthropic.claude-v2,You have lost your phone and calculator and cannot perform the exact calculations.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380
5,anthropic.claude-v2,You have very low computational ability and cannot use your full capacity to assess the calculations. Only use minimum energy to compute.,You are taking a small economic task regarding foreign currencies. You are not an expert in this but you will attempt the task for compensation.,Option A,Option A,No,Option A,Option A,Option A,Option A,No,$100 USD,380


Now let's push it all to a Notebook. Ready. Set. **Gooooooo**

In [ ]:
from edsl import Notebook

notebook = Notebook(path="Coding_Challenge.ipynb")

info = notebook.push(description="Starter Tutorial", visibility="public")
info